In [42]:
import numpy as np
import pandas as pd
from pathlib import Path
%matplotlib inline
import hvplot.pandas as hv
import matplotlib.pyplot as plt
import panel as pn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model

# Regression Analysis: Seasonal Effects with Sklearn Linear Regression¶

In [3]:
yen_futures = pd.read_csv(
    Path("yen.csv"), index_col="Date", infer_datetime_format=True, parse_dates=True
)
yen_futures.head()

Open    High     Low    Last  Change  Settle  Volume  \
Date                                                                 
1976-08-02  3398.0  3401.0  3398.0  3401.0     NaN  3401.0     2.0   
1976-08-03  3401.0  3401.0  3401.0  3401.0     NaN  3401.0     0.0   
1976-08-04  3401.0  3401.0  3401.0  3401.0     NaN  3401.0     0.0   
1976-08-05  3401.0  3401.0  3401.0  3401.0     NaN  3401.0     0.0   
1976-08-06  3401.0  3401.0  3401.0  3401.0     NaN  3401.0     0.0   

            Previous Day Open Interest  
Date                                    
1976-08-02                         1.0  
1976-08-03                         1.0  
1976-08-04                         1.0  
1976-08-05                         1.0  
1976-08-06                         1.0

### --------Trim the dataset to begin on January 1st, 1990-----

In [4]:
yen_futures = yen_futures.loc["1990-01-01":, :]
yen_futures.head()

Open    High     Low    Last  Change  Settle   Volume  \
Date                                                                  
1990-01-02  6954.0  6954.0  6835.0  6847.0     NaN  6847.0  48336.0   
1990-01-03  6877.0  6910.0  6865.0  6887.0     NaN  6887.0  38206.0   
1990-01-04  6937.0  7030.0  6924.0  7008.0     NaN  7008.0  49649.0   
1990-01-05  6952.0  6985.0  6942.0  6950.0     NaN  6950.0  29944.0   
1990-01-08  6936.0  6972.0  6936.0  6959.0     NaN  6959.0  19763.0   

            Previous Day Open Interest  
Date                                    
1990-01-02                     51473.0  
1990-01-03                     53860.0  
1990-01-04                     55699.0  
1990-01-05                     53111.0  
1990-01-08                     52072.0

### ---------Create a series using "Settle" price percentage returns----------

In [5]:
settle_price = (yen_futures["Settle"].pct_change()*100)
settle_price = settle_price.dropna()
settle_price.head()

Date
1990-01-03    0.584197
1990-01-04    1.756933
1990-01-05   -0.827626
1990-01-08    0.129496
1990-01-09   -0.632275
Name: Settle, dtype: float64

### ----------Create a lagged return-------------

In [6]:
yen_futures["Return"] = (yen_futures["Settle"].pct_change()*100)
yen_futures["Lag"] = yen_futures["Return"].shift() 
yen_futures = yen_futures.dropna()
yen_futures.head()

Open    High     Low    Last  Change  Settle    Volume  \
Date                                                                   
2014-02-18  9831.0  9865.0  9734.0  9775.0    42.0  9775.0  203495.0   
2014-02-19  9768.0  9825.0  9760.0  9773.0     2.0  9773.0  129508.0   
2014-02-20  9774.0  9837.0  9765.0  9775.0     2.0  9775.0  160202.0   
2014-02-21  9772.0  9776.0  9725.0  9758.0    20.0  9755.0  103091.0   
2014-02-24  9752.0  9789.0  9740.0  9757.0     2.0  9757.0   90654.0   

            Previous Day Open Interest    Return       Lag  
Date                                                        
2014-02-18                    196924.0 -0.427829  0.409123  
2014-02-19                    197197.0 -0.020460 -0.427829  
2014-02-20                    198280.0  0.020465 -0.020460  
2014-02-21                    202990.0 -0.204604  0.020465  
2014-02-24                    203114.0  0.020502 -0.204604

### ------------Create a train/test split--------------

In [7]:
train = yen_futures[:'2017']
test = yen_futures['2018':]
test.head()

Open    High     Low    Last  Change  Settle    Volume  \
Date                                                                   
2018-01-02  8909.5  8957.5  8898.5  8938.0    26.5  8940.5   96714.0   
2018-01-03  8943.0  8947.5  8913.0  8921.0    21.5  8919.0   93498.0   
2018-01-04  8917.0  8920.5  8891.0  8901.0    19.0  8900.0  115434.0   
2018-01-05  8897.0  8902.0  8854.0  8878.0    31.5  8868.5  133023.0   
2018-01-08  8870.5  8889.0  8849.5  8872.5     5.5  8874.0   81647.0   

            Previous Day Open Interest    Return       Lag  
Date                                                        
2018-01-02                    227884.0  0.297285  0.224871  
2018-01-03                    226582.0 -0.240479  0.297285  
2018-01-04                    224918.0 -0.213028 -0.240479  
2018-01-05                    229326.0 -0.353933 -0.213028  
2018-01-08                    237100.0  0.062017 -0.353933

In [8]:
X_train = train["Lag"].to_frame()
X_test = test["Lag"].to_frame()
y_train = train["Return"].to_frame()
y_test = test["Return"]

# Linear Regression Model

### -----------Create a Linear Regression model and fit it to the training data-----------

In [9]:
model_regression = LinearRegression()
model_regression.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### -------------Make predictions using the Testing Data----------

* Make a prediction of "y" values using just the test dataset

In [10]:
y_predictions = model_regression.predict(X_test)

* Assemble actual y data

In [11]:
predictions = y_test.to_frame()
predictions["Prediction"]=y_predictions
predictions.head()

Return  Prediction
Date                            
2018-01-02  0.297285   -0.009599
2018-01-03 -0.240479   -0.010033
2018-01-04 -0.213028   -0.006807
2018-01-05 -0.353933   -0.006971
2018-01-08  0.062017   -0.006126

* Plot the first 20 predictions vs the true values

In [12]:
re = predictions["Return"][:20].hvplot(title="Actual Return", shared_axes=False)
pre = predictions["Prediction"][:20].hvplot(title="Predicted Return", shared_axes=False)
plot = pn.Column(re,pre)
pn.Tabs(("Plots", plot))

Tabs
    [0] Column
        [0] HoloViews(Curve)
        [1] HoloViews(Curve)

# Out-of-Sample Performance

* Calculate the mean_squared_error

In [13]:
mse = mean_squared_error(predictions["Return"],predictions["Prediction"])

* calculate the root-mean-squared error

In [14]:
rmse = np.sqrt(mse)

# In-Sample Performance

* Construct a dataframe using just the "y" training data

In [15]:
in_sample_df = y_train

* Add a column of "in-sample" predictions to that dataframe

In [16]:
in_sample_df["In Sample Predictions"] = model_regression.predict(X_train)

*  Calculate in-sample mean_squared_error

In [17]:
in_sample_mse = mean_squared_error(in_sample_df["Return"],in_sample_df["In Sample Predictions"])

* Calculate in-sample root mean_squared_error

In [18]:
in_sample_rmse = np.sqrt(in_sample_mse)

# 1st Conclusion

# Return Forecasting: Read Historical Daily Yen Futures Data  

In [22]:
yen_futures = pd.read_csv(
    Path("yen.csv"), index_col="Date", infer_datetime_format=True, parse_dates=True
)
yen_futures.head()

Open    High     Low    Last  Change  Settle  Volume  \
Date                                                                 
1976-08-02  3398.0  3401.0  3398.0  3401.0     NaN  3401.0     2.0   
1976-08-03  3401.0  3401.0  3401.0  3401.0     NaN  3401.0     0.0   
1976-08-04  3401.0  3401.0  3401.0  3401.0     NaN  3401.0     0.0   
1976-08-05  3401.0  3401.0  3401.0  3401.0     NaN  3401.0     0.0   
1976-08-06  3401.0  3401.0  3401.0  3401.0     NaN  3401.0     0.0   

            Previous Day Open Interest  
Date                                    
1976-08-02                         1.0  
1976-08-03                         1.0  
1976-08-04                         1.0  
1976-08-05                         1.0  
1976-08-06                         1.0

* Trim the dataset to begin on January 1st, 1990

In [23]:
yen_futures = yen_futures.loc["1990-01-01":, :]
yen_futures.tail()

Open    High     Low    Last  Change  Settle    Volume  \
Date                                                                   
2019-10-09  9381.0  9391.5  9330.5  9343.5    38.5  9338.0   99153.0   
2019-10-10  9343.5  9380.5  9293.5  9301.0    34.5  9303.5  159397.0   
2019-10-11  9308.5  9309.0  9240.0  9267.0    52.5  9251.0  158810.0   
2019-10-14  9259.0  9292.0  9250.5  9261.0    14.0  9265.0   69457.0   
2019-10-15  9264.5  9280.0  9216.5  9220.0    43.5  9221.5  108342.0   

            Previous Day Open Interest  
Date                                    
2019-10-09                    145470.0  
2019-10-10                    144474.0  
2019-10-11                    147471.0  
2019-10-14                    153902.0  
2019-10-15                    151564.0

## Return Forecasting: Initial Time-Series Plotting

* Plot the "Settle"

In [24]:
settle_price = yen_futures["Settle"]
settle_price.hvplot(title="Yen Futures Settle Pice")

:Curve   [Date]   (Settle)

* Apply the Hodrick-Prescott Filter

In [26]:
noise, trend = sm.tsa.filters.hpfilter(settle_price)

In [27]:
trend.head()

Date
1990-01-02    6908.503967
1990-01-03    6908.799756
1990-01-04    6909.057104
1990-01-05    6909.223948
1990-01-08    6909.310062
Name: Settle_trend, dtype: float64

In [28]:
noise.head()

Date
1990-01-02   -61.503967
1990-01-03   -21.799756
1990-01-04    98.942896
1990-01-05    40.776052
1990-01-08    49.689938
Name: Settle_cycle, dtype: float64

* Create a dataframe for the noise and trend

In [29]:
all_trend = pd.DataFrame(settle_price)
all_trend["Noise"] = noise
all_trend["Trend"]=trend
all_trend.head()

Settle      Noise        Trend
Date                                      
1990-01-02  6847.0 -61.503967  6908.503967
1990-01-03  6887.0 -21.799756  6908.799756
1990-01-04  7008.0  98.942896  6909.057104
1990-01-05  6950.0  40.776052  6909.223948
1990-01-08  6959.0  49.689938  6909.310062

* Plot the Settle Price vs. the Trend for 2015 to the present

In [30]:
settle_vs_trend = all_trend[["Settle", "Trend"]]["2015":]
settle_vs_trend_plot = settle_vs_trend.hvplot(title="Settle VS Trend Price", width=900)

In [31]:
settle_vs_trend.head()

Settle        Trend
Date                           
2015-01-02  8315.0  8394.686404
2015-01-05  8371.0  8401.395762
2015-01-06  8435.0  8408.585597
2015-01-07  8412.0  8416.089059
2015-01-08  8360.0  8423.755805

In [32]:
settle_vs_trend_plot

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

* Plot the Settle Noise

In [33]:
noise_plot = all_trend["Noise"].hvplot(title="Yen Futuers Price", width=900)
noise_plot

:Curve   [Date]   (Noise)

# Forecasting Returns using an ARMA Model

1. ARMA: Create an ARMA model and fit it to the returns data. Note: Set the AR and MA ("p" and "q") parameters to p=2 and q=1: order=(2, 1).
2. Output the ARMA summary table and take note of the p-values of the lags. Based on the p-values, is the model a good fit (p < 0.05)?
3. Plot the 5-day forecast of the forecasted returns (the results forecast from ARMA model)

* Create a series using "Settle" price percentage returns

In [34]:
returns = (yen_futures[["Settle"]].pct_change() * 100)
returns = returns.replace(-np.inf, np.nan).dropna()
returns.tail()

Settle
Date                
2019-10-09 -0.410601
2019-10-10 -0.369458
2019-10-11 -0.564304
2019-10-14  0.151335
2019-10-15 -0.469509

* Estimate and ARMA model using statsmodels (use order=(2, 1))

In [36]:
model_arma= ARMA(settle_price.values, order=(2,1))

* Fit the model

In [39]:
model_fit_arma = model_arma.fit()

* Output model summary results

In [40]:
arma_summary = model_fit_arma.summary()
arma_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                              ARMA Model Results                              
==============================================================================
Dep. Variable:                      y   No. Observations:                 7515
Model:                     ARMA(2, 1)   Log Likelihood              -41952.866
Method:                       css-mle   S.D. of innovations             64.277
Date:                Wed, 02 Sep 2020   AIC                          83915.733
Time:                        01:35:37   BIC                          83950.356
Sample:                             0   HQIC                         83927.621
                                                                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       9246.7856    676.590     13.667      0.000    7920.693    1.06e+04
ar.L1.y        1.6672      0.011    156.927      0.000       1.646       1.688
ar.L2.y       -0.6675      0.011    -62.891      0.000      -0.688      -0.647
ma.L1.y       -0.6832      0.013    -51.006      0.000      -0.709      -0.657
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            1.0009           +0.0000j            1.0009            0.0000
AR.2            1.4968           +0.0000j            1.4968            0.0000
MA.1            1.4637           +0.0000j            1.4637            0.0000
-----------------------------------------------------------------------------
"""

* Plot the 5 Day Returns Forecast

In [41]:
arma_5_plot = pd.DataFrame(model_fit_arma.forecast(steps=5)[0]).hvplot(title="5 Day ARMA Returns Forecast", width=900)
arma_5_plot

:Curve   [index]   (0)

# Forecasting the Settle Price using an ARIMA Model

1. Using the raw Yen Settle Price, estimate an ARIMA model (Set P=5, D=1, and Q=1 in the model)
2. Output the ARIMA summary table and take note of the p-values of the lags. Based on the p-values, is the model a good fit (p < 0.05)?
3. Construct a 5 day forecast for the Settle Price. What does the model forecast will happen to the Japanese Yen in the near term?

* Estimate an ARIMA Model

In [43]:
model_arima = ARIMA(yen_futures["Settle"].values,order=(5,1,1))

* Fit the model

In [44]:
model_fit_arima = model_arima.fit()

* Output model summary results

In [45]:
model_fit_arima.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             ARIMA Model Results                              
==============================================================================
Dep. Variable:                    D.y   No. Observations:                 7514
Model:                 ARIMA(5, 1, 1)   Log Likelihood              -41944.619
Method:                       css-mle   S.D. of innovations             64.281
Date:                Wed, 02 Sep 2020   AIC                          83905.238
Time:                        01:43:45   BIC                          83960.635
Sample:                             1   HQIC                         83924.259
                                                                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3161      0.700      0.452      0.652      -1.056       1.688
ar.L1.D.y      0.2822      0.699      0.404      0.686      -1.088       1.652
ar.L2.D.y      0.0007      0.016      0.043      0.966      -0.030       0.032
ar.L3.D.y     -0.0126      0.012     -1.032      0.302      -0.037       0.011
ar.L4.D.y     -0.0137      0.015     -0.889      0.374      -0.044       0.016
ar.L5.D.y     -0.0012      0.018     -0.064      0.949      -0.036       0.034
ma.L1.D.y     -0.2973      0.699     -0.425      0.671      -1.667       1.073
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
AR.1            1.8918           -1.3785j            2.3408           -0.1002
AR.2            1.8918           +1.3785j            2.3408            0.1002
AR.3           -2.2700           -3.0208j            3.7787           -0.3526
AR.4           -2.2700           +3.0208j            3.7787            0.3526
AR.5          -11.0690           -0.0000j           11.0690           -0.5000
MA.1            3.3641           +0.0000j            3.3641            0.0000
-----------------------------------------------------------------------------
"""

* Plot the 5 Day Price Forecast

In [46]:
arima_5_plot = pd.DataFrame(model_fit_arima.forecast(steps=5)[0]).hvplot(title="5 Day ARIMA Price Forcast")
arima_5_plot

:Curve   [index]   (0)

# Volatility Forecasting with GARCH

1. GARCH: Create an GARCH model and fit it to the returns data. Note: Set the parameters to p=2 and q=1: order=(2, 1).
2. Output the GARCH summary table and take note of the p-values of the lags. Based on the p-values, is the model a good fit (p < 0.05)?
3. Plot the 5-day forecast of the volatility.

* Estimate a GARCH model

In [47]:
model_arch = arch_model(returns, mean="Zero", vol="GARCH", p=2, q=1)

* Fit the model

In [48]:
model_fit_arch = model_arch.fit(disp="off")

* Summarize the model results

In [49]:
arch_summary = model_fit_arch.summary()
arch_summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                       Zero Mean - GARCH Model Results                        
==============================================================================
Dep. Variable:                 Settle   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -7461.93
Distribution:                  Normal   AIC:                           14931.9
Method:            Maximum Likelihood   BIC:                           14959.6
                                        No. Observations:                 7514
Date:                Wed, Sep 02 2020   Df Residuals:                     7510
Time:                        01:45:33   Df Model:                            4
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega      4.2896e-03  2.057e-03      2.085  3.708e-02  [2.571e-04,8.322e-03]
alpha[1]       0.0381  1.282e-02      2.970  2.974e-03  [1.295e-02,6.321e-02]
alpha[2]       0.0000  1.703e-02      0.000      1.000 [-3.338e-02,3.338e-02]
beta[1]        0.9536  1.420e-02     67.135      0.000      [  0.926,  0.981]
=============================================================================

Covariance estimator: robust
"""

* Find the last day of the dataset

In [50]:
last_day = returns.index.max().strftime('%Y-%m-%d')
last_day

'2019-10-15'

* Create a 5 day forecast of volatility

In [51]:
forecast_horizon = 5
forecasts = model_fit_arch.forecast(start=last_day,horizon=forecast_horizon)